<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/BUG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/intelligence-csd-auth-gr/Ethos-Hate-Speech-Dataset.git
!pip install -qq transformers


Cloning into 'Ethos-Hate-Speech-Dataset'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 93 (delta 41), reused 52 (delta 13), pack-reused 0
Unpacking objects: 100% (93/93), done.
     |████████████████████████████████| 1.8MB 8.4MB/s 
     |████████████████████████████████| 890kB 34.6MB/s 
     |████████████████████████████████| 2.9MB 54.4MB/s 


In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
data_path='Ethos-Hate-Speech-Dataset/ethos/ethos_data/Ethos_Dataset_Binary.csv'
df = pd.read_csv(data_path, delimiter=';')
df['id'] = range(len(df))
df.set_index('id', inplace=True)
df.head()

,comment,isHate
id,,
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0


In [4]:
df.isHate.value_counts()

0.000000    354
1.000000    163
0.166667    106
0.833333    100
0.333333     80
0.500000     74
0.666667     70
0.250000      6
0.750000      6
0.857143      3
0.018868      2
0.400000      2
0.903846      2
0.103448      1
0.846154      1
0.978261      1
0.152542      1
0.031746      1
0.603448      1
0.302326      1
0.937500      1
0.849057      1
0.030303      1
0.296875      1
0.111111      1
0.722222      1
0.160714      1
0.016393      1
0.029851      1
0.038961      1
0.200000      1
0.954545      1
0.945455      1
0.983871      1
0.026316      1
0.966667      1
0.678571      1
0.821429      1
0.983607      1
0.973333      1
0.037736      1
0.090909      1
0.530612      1
Name: isHate, dtype: int64

In [5]:
def to_category(score):
  if score <= 0.5:
    return 'NO'
  elif score>0.5:
    return 'YES'

df['isHate_category']=df.isHate.apply(to_category)
df.head()

,comment,isHate,isHate_category
id,,,
0,You should know women's sports are a joke,1.0,YES
1,You look like Sloth with deeper Down’s syndrome,1.0,YES
2,You look like Russian and speak like Indian. B...,1.0,YES
3,"Women deserve to be abused, I guess.",1.0,YES
4,Women are made for making babies and cooking d...,1.0,YES


In [6]:
df.isHate_category.value_counts()

NO     639
YES    359
Name: isHate_category, dtype: int64

In [ ]:
df.isHate_category.value_counts()

NO     482
YES    286
MID    230
Name: isHate_category, dtype: int64

In [7]:
possible_labels=df.isHate_category.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NO': 1, 'YES': 0}

In [8]:
df['label']=df.isHate_category.replace(label_dict)
df.head()

,comment,isHate,isHate_category,label
id,,,,
0,You should know women's sports are a joke,1.0,YES,0
1,You look like Sloth with deeper Down’s syndrome,1.0,YES,0
2,You look like Russian and speak like Indian. B...,1.0,YES,0
3,"Women deserve to be abused, I guess.",1.0,YES,0
4,Women are made for making babies and cooking d...,1.0,YES,0


In [10]:
!pip install emoji --upgrade
!pip install wordsegment
from wordsegment import load
load()

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.6.0)
     |████████████████████████████████| 4.8MB 7.5MB/s 


In [21]:
from wordsegment import segment
import emoji
import re
def clean(word):
  
  text = re.sub(r"can't", "can not ", word)
  text = re.sub(r"don't", "can not ", text)
  text = re.sub(r"\'s", "", word)
  text = re.sub(r"\'ve", "have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'m", "am", text)
  text = re.sub(r"%", "percent", text) 
  return text 

def foremoji(text):
  text = re.sub(r"’", "'", text)
  text = re.sub(r":", " : ", text)
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text: 
    w=clean(w)  
    w=re.sub(r":", " <emoji> ", w)
    w=re.sub(r"@USER","",w)
    w=re.sub(r"_"," ",w)
    if re.match("^#",w):
      w=" ".join(segment(w))
    new_text.append(w)
  text = " ".join(new_text)
  return "<real> "+text

In [22]:
df['emoji_tweet']=df.comment.apply(foremoji)

In [23]:
df

,comment,isHate,isHate_category,label,emoji_tweet
id,,,,,
0,You should know women's sports are a joke,1.0,YES,0,<real> You should know women sports are a joke
1,You look like Sloth with deeper Down’s syndrome,1.0,YES,0,<real> You look like Sloth with deeper Down sy...
2,You look like Russian and speak like Indian. B...,1.0,YES,0,<real> You look like Russian and speak like In...
3,"Women deserve to be abused, I guess.",1.0,YES,0,"<real> Women deserve to be abused, I guess."
4,Women are made for making babies and cooking d...,1.0,YES,0,<real> Women are made for making babies and co...
...,...,...,...,...,...
993,From the midnight sun where the hot springs blow,0.0,NO,1,<real> From the midnight sun where the hot spr...
994,Don't say I'm not your type,0.0,NO,1,<real> Don't say Iam not your type
995,And therefore never send to know for whom the...,0.0,NO,1,<real> And therefore never send to know for wh...


In [24]:
test=df

In [26]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [27]:
tokenizer.add_special_tokens({'additional_special_tokens':["<emoji>","<real>","<fake>"]})

3

In [30]:
test['data_type']=['test']*test.shape[0]
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
encoder_data_test=tokenizer.batch_encode_plus(
    test[test.data_type=='test'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)
inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(test[test.data_type=='test'].label.values)
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [32]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat, digits=5)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [35]:
from transformers import BertForSequenceClassification


model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [36]:
model.resize_token_embeddings(len(tokenizer))

Embedding(28999, 768)

In [40]:

model.load_state_dict(torch.load('/content/augmentation.model', map_location=torch.device('cuda')))


RuntimeError: ignored

In [ ]:
df.loc[X_train,'data_type']='train'
# df.loc[df_val,'data_type']='val'
df.loc[X_val,'data_type']='val'

In [ ]:
df.head()

,comment,isHate,isHate_category,label,data_type
id,,,,,
0,You should know women's sports are a joke,1.0,YES,0,train
1,You look like Sloth with deeper Down’s syndrome,1.0,YES,0,train
2,You look like Russian and speak like Indian. B...,1.0,YES,0,val
3,"Women deserve to be abused, I guess.",1.0,YES,0,train
4,Women are made for making babies and cooking d...,1.0,YES,0,train


In [ ]:
df.data_type.value_counts()

not_set    995
test         3
Name: data_type, dtype: int64

In [ ]:
df.groupby(['isHate_category','label','data_type']).count()

comment  isHate
isHate_category label data_type                 
MID             1     train          184     184
                      val             46      46
NO              2     train          385     385
                      val             97      97
YES             0     train          229     229
                      val             57      57

In [ ]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
encoder_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].comment.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

encoder_data_val=tokenizer.batch_encode_plus(
    df[df.data_type=='val'].comment.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation =True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

inputs_ids_train=encoder_data_train['input_ids']
attentions_masks_train=encoder_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].label.values)

inputs_ids_val=encoder_data_val['input_ids']
attentions_masks_val=encoder_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
from torch.utils.data import TensorDataset
dataset_train=TensorDataset(inputs_ids_train,
              attentions_masks_train, labels_train)
dataset_val=TensorDataset(inputs_ids_val,
              attentions_masks_val, labels_val)

In [ ]:
len(dataset_train)

798

In [ ]:
len(dataset_val)

200

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
BATCH_SIZE=16

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=BATCH_SIZE
)

dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train)*epochs)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.8992537558078766
Validation loss: 0.8642627826103797
F1 Score (Weighted): 0.5536934853995268



Epoch 2
Training loss: 0.7334724390506744
Validation loss: 0.803472828406554
F1 Score (Weighted): 0.5958165154183034



Epoch 3
Training loss: 0.5791224730014801
Validation loss: 0.7996163918421819
F1 Score (Weighted): 0.6346739033283096



Epoch 4
Training loss: 0.4863027992844582
Validation loss: 0.7706979948740739
F1 Score (Weighted): 0.6488530219780221



Epoch 5
Training loss: 0.4010874384641647
Validation loss: 0.8326604549701397
F1 Score (Weighted): 0.6350295075010125



Epoch 6
Training loss: 0.3511258244514465
Validation loss: 0.8040269005757111
F1 Score (Weighted): 0.6273779152171112



Epoch 7
Training loss: 0.31643260896205905
Validation loss: 0.8785429092553946
F1 Score (Weighted): 0.6525084233415545



Epoch 8
Training loss: 0.29152686476707457
Validation loss: 0.8364941294376667
F1 Score (Weighted): 0.6641230237810549



Epoch 9
Training loss: 0.27016456320881843
Validation loss: 0.851364445227843
F1 Score (Weighted): 0.6472902057374674



Epoch 10
Training loss: 0.26408902421593666
Validation loss: 0.8414036127237173
F1 Score (Weighted): 0.6472902057374674



In [ ]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_8.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:

_, predictions, true_vals = evaluate(dataloader_val)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: YES
Accuracy: 38/57

Class: MID
Accuracy: 18/46

Class: NO
Accuracy: 77/97

